In [1]:
import pandas
import requests
from GIS_Data_Library_ext import *
import os
from databaseUpdaterGIM import *

In [2]:
oe_api_key = os.environ['OE_API_KEY']

# Bring in data from OE databanks
Find saved databank selections

In [3]:
gem_selection = find_selection(oe_api_key, 'PPI_GEM_Q')
nac_selection = find_selection(oe_api_key, 'PPI_NAC_Q')
gis_selection = find_selection(oe_api_key, 'PPI_GIS_Q')
usind_selection = find_selection(oe_api_key, 'PPI_USIND_Q')

In [4]:
selections = [gem_selection, nac_selection, gis_selection, usind_selection]
databanks = ['gem', 'nac', 'gis', 'usind']

In [5]:
selection_dict = dict(zip(databanks, selections))

Download databank selections

In [6]:
data_main = {}
metadata_main = {}

for i in selection_dict.keys():
    # Pull Global Economic Service data
    response = requests.get("https://services.oxfordeconomics.com/api/download/" + selection_dict[i],
                            headers = {'Accept' : 'application/json', 
                                        'API-Key' : oe_api_key,
                                        'Content-Type' : 'application/json; charset=utf8'})

    data_pull = response.json()

    # extract mnemonics, quarterly data and metadata
    data = {}
    metadata = {}
    for j in range(0, len(data_pull)):
        vbl = data_pull[j]['VariableCode']
        data_ts = data_pull[j]['QuarterlyData']
        metadata = data_pull[j]['Metadata']
        data[vbl] = data_ts
        metadata[vbl] = metadata

    data_main[i] = data
    metadata_main[i] = metadata

Convert to dataframes

In [ ]:
pd.DataFrame(data_main['gis'])

,GYA,GYE,GYEXOIL,GYCOAL,GYOTHE,GYOIL,GYM,GYFDBVT,GYFDBV,GYFD,...,PRHEAL,PROTHS,PRT,PRCONN,PRCON,PRINT,PRINV,PRENG,PRCOND,PRALL
198001,135.3213,165.3582,79.29815,5.318923,73.97923,86.06002,1176.277,130.3389,96.06960,80.21888,...,5.616665,5.941009,77.72195,13.29738,15.54806,19.57521,14.24884,8.676302,2.250688,258.0333
198002,124.3164,182.9442,88.91398,5.574709,83.33927,94.03027,1200.222,129.9507,95.90181,80.15729,...,5.778865,6.095893,79.49569,13.95140,16.17252,19.22124,14.20159,8.608751,2.221118,263.0093
198003,118.8226,198.4818,97.36706,5.634972,91.73209,101.11470,1195.531,132.5262,97.52406,81.66309,...,5.973633,6.264984,82.15466,14.42561,16.67453,19.55928,14.52054,8.659698,2.248926,270.6555
198004,119.0792,212.1210,104.62930,6.093189,98.53609,107.49180,1161.527,135.5237,98.22384,82.03096,...,6.200970,6.448282,85.13618,14.59359,16.89598,20.38818,15.00226,8.837729,2.302387,280.4093
198101,120.7879,234.5122,116.93930,7.088088,109.85120,117.57290,1161.963,144.8580,104.90540,88.02969,...,6.437149,6.670292,89.34497,15.18842,17.62085,21.56975,15.44726,9.217494,2.432428,292.3469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204904,128.9777,110.5001,29.53208,0.482634,29.04944,80.96803,2156.373,355.5832,347.15370,296.60930,...,185.950100,115.245600,1517.40700,301.23310,342.28380,324.13810,293.57910,181.397800,41.050700,6880.2250
205001,128.9624,110.1606,29.37095,0.475296,28.89566,80.78964,2163.331,356.4805,348.12350,297.53610,...,187.312700,115.931000,1530.50300,304.20440,345.59880,326.57850,296.56770,183.581500,41.394420,6940.3700
205002,128.9489,109.8208,29.20908,0.468096,28.74099,80.61173,2170.306,357.3798,349.09490,298.46470,...,188.679400,116.617700,1543.47100,307.18050,348.90170,329.02740,299.44030,185.756300,41.721180,7000.7010
205003,128.9341,109.4887,29.04647,0.461032,28.58544,80.44226,2177.303,358.2580,350.04520,299.37600,...,190.057200,117.309500,1556.88500,310.21630,352.28590,331.54250,302.49240,187.992100,42.069550,7061.9320


In [ ]:
dataframes = {}

In [ ]:
for i in data_main.keys():
    dataframes[i] = pd.DataFrame(data_main[i])
    dataframes[i].index = pd.period_range(start = '1980-01-01', end = '2050-12-31', freq = 'Q')

In [ ]:
# calculate profit margin by industy as PRx/GYx!
gis_sectors = ['A', 'E', 'EXOIL', 'COAL', 'OTHE', 'OIL', 'M', 'FDBVT', 'FDBV', 'FD', 'BV', 'TOBC', 'TEXT', 'TETEX', 'TECLO', 'TELEA', 'WOOD', 'PAPR', 'PAPB', 'PAPT', 'FCHR', 'FUEL', 'CHEM', 'CHXPHA', 'CHBAS', 'CHAGR', 'CHPNT', 'CHSOP', 'CHOTH', 'CHMMF', 'CHPHA', 'RUBP', 'MINL', 'MLGL', 'MLCER', 'MLCSN', 'BMETP', 'BMET', 'BMIST', 'BMNFR', 'BMCST', 'METP', 'ELO', 'ELC', 'COMP', 'ELEC', 'ELEC32', 'ELCMP', 'ELTEL', 'ELCSM', 'PREC', 'PRMED', 'PROTH', 'ELEC31', 'ELPWR', 'MEDOM', 'ELDIS', 'ELOTH', 'MECH', 'MEGPM', 'MEPWR', 'MEOGN', 'MESPM', 'MEAGR', 'MEMAC', 'MEOTH', 'MEOTHMIN', 'MOTRA', 'MOTR', 'MOVEH', 'MOPRT', 'OTRA', 'OTAER', 'OTNES', 'OTHF', 'OTHM', 'REPAIR', 'U', 'UEGAS', 'ELECTR', 'UGAS', 'UWSWM', 'P', 'C', 'S', 'DTRC', 'DIS', 'DISTR', 'HOTL', 'TRACOM', 'TRA', 'TRALRP', 'TRAW', 'TRAA', 'TRAOTH', 'COM', 'COMPUB', 'COMTEL', 'COMIT', 'FIBU', 'FIN', 'BUS', 'REAL', 'BUSOTH', 'OTH', 'PAD', 'PUBLIC', 'EDUC', 'HEAL', 'OTHS', 'T', 'CONN', 'CON', 'INT', 'INV', 'ENG', 'COND', 'ALL']

for i in gis_sectors:
    if i == 'ALL':
        dataframes['gis']['PM'+i] = dataframes['gis']['PR'+i] / dataframes['gis']['GY'+'GVA'+'!']
    else:
        dataframes['gis']['PM'+i] = dataframes['gis']['PR'+i] / dataframes['gis']['GY'+i+'!']

C:\Users\PETERC~1\AppData\Local\Temp/ipykernel_26616/1966365235.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframes['gis']['PM'+i] = dataframes['gis']['PR'+i] / dataframes['gis']['GY'+i+'!']
C:\Users\PETERC~1\AppData\Local\Temp/ipykernel_26616/1966365235.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframes['gis']['PM'+i] = dataframes['gis']['PR'+i] / dataframes['gis']['GY'+'GVA'+'!']


In [ ]:
for i in dataframes.keys():
    dataframes[i].to_csv(i+"_covariates.csv")

In [ ]:
dataframes.keys()

dict_keys(['gem', 'nac', 'gis', 'usind'])

# Pull COSTx and PENGx variables from the current internal database

In [2]:
# dictionary Country : Variables
sel_vbls = {'US' : ['PENGIND',
                    'COSTBMMET',
                    'COSTBMNFR',
                    'COSTCHEM',
                    'COSTCOM',
                    'COSTELC',
                    'COSTELEC31',
                    'COSTFDBVT',
                    'COSTFUEL',
                    'COSTMECH',
                    'COSTMETP',
                    'COSTMINL',
                    'COSTMOTR',
                    'COSTOTHFMRE',
                    'COSTOTHM',
                    'COSTOTRA',
                    'COSTPAPR',
                    'COSTRUBP',
                    'COSTTEXT',
                    'COSTTOT',
                    'COSTTRAA',
                    'COSTTRALRP',
                    'COSTTRAOTH',
                    'COSTTRAW',
                    'COSTU',
                    'COSTWOOD',
                    'PENGTOT',
                    'PENGA',
                    'PENGBMMET',
                    'PENGBMNFR',
                    'PENGC',
                    'PENGCHEM',
                    'PENGCOMMER',
                    'PENGE',
                    'PENGELECTR',
                    'PENGFDBVT',
                    'PENGFUEL',
                    'PENGMINL',
                    'PENGMMAC',
                    'PENGMOTH',
                    'PENGMOTRA',
                    'PENGPAPR',
                    'PENGPUBLIC',
                    'PENGTEXT',
                    'PENGTRAA',
                    'PENGTRALRP',
                    'PENGTRAOTH',
                    'PENGTRAW',
                    'PENGUGAS',
                    'PENGWOOD']}

start = '1980'
end = '2050'
var = True
agg = 'L'

In [3]:
cost_peng_df = databaseUpdater(start, end, sel_vbls, var, agg)

sel file created in c:\DSWG_PPI\Databases\GIM\


In [ ]:
cost_peng_df.columns = cost_peng_df.loc[2]
cost_peng_df = cost_peng_df.drop([0,1,2,3,4,5,6,7,8])

In [ ]:
cost_peng_df.index = pd.period_range(start=start+'-01-01', end=end+'-12-31', freq='Q')

In [ ]:
cost_peng_df.to_csv('cost_peng_covariates.csv')

# Combine all covariates files

In [ ]:
all_covariates = pd.concat([dataframes['gem'],dataframes['nac'],dataframes['usind'],dataframes['gis'],cost_peng_df],axis=1)

In [ ]:
all_covariates.to_csv('all_covariates.csv')